In [ ]:
import numpy as np
import cv2 as cv 
import os #manipular archivos q tengamos en carpeta especifica, dar ruta, donde tenemos nuestras img


In [ ]:
#GENERAR MODELO, por caras de personas

# Folder de caras, dentro hay otro folder "actividad2", dentro hay mas folders, de cada persona, o de cada sentimiento, para que ponga etiquetas a cada uno
dataSet =  r'C:\Users\rovega\Desktop\ITM\Sem 9\Inteligencia Artificial\IA - Actividades NO GIT\caras\actividad2'
faces = os.listdir(dataSet) #Guarda en una lista, el nombre de las carpetas de donde se va a entrenar
print(faces) #Imprime lista de carpetas

# Lista para guardar las etiquetas 
labels = [] #lista de labels , cuando encuentra carpeta, le pone una etiqueta, si encuentra otra carpeta le pone otra etiqueta.
#aprendizaje supervisado: esta cara es mia de roy, y le pone etiquetas
#aprendizja no supervisado: tendria que adivinar las img

# Lista para guardar los rostros
facesData = [] #fisherface necesita dos carpetas (dos etiquetas)
label = 0

#si tengo un directorio con caras, recorro cada uno y cada uno de ellos pertence a esos conjuntos
#cada carpeta [roy] [pepe] [juan] la recorro 
for face in faces: #todo este conjunto pertence a etiqueta 1, genera labels a cada conjunto de img
    
    facePath = dataSet+'\\'+face #muevo por directorio
    #facePath = os.path.join(dataSet, face)
    
    if os.path.isdir(facePath):  # Check if it's a directory
        
        for faceName in os.listdir(facePath): #Itera sobre rostros de cada carpeta de cada persona 
            labels.append(label)
            facesData.append(cv.imread(facePath+'/'+faceName,0)) # agregar la img del rostro y su etiqeuta correspondiendo a la lista
        label = label + 1 #incrementar etiqueta para sig persona.
print(np.count_nonzero(np.array(labels)==0))

faceRecognizer = cv.face.EigenFaceRecognizer_create()

 #TARDA EN ENTRENAR 
faceRecognizer.train(facesData, np.array(labels))#a traves de este conjunto de eitqeuta sempeiza a entrenar, no puedo tener caras de otra persona en mi carpeta, cuando intente separar info, no va a saber.
faceRecognizer.write(r'C:\Users\rovega\Desktop\ITM\Sem 9\Inteligencia Artificial\Git IA - Actividades\ModelosXML\Eigenface-carasSentimientos.xml') #guardar modelo entrenado como xml
print('Exito!')
#GENERA XML YA ENTRENADO 
#POR CADA CARA, GENERA DATASET= TODA LA CARPETA DE CARA ROY ETIQEUTA 0, PEPE ETIQUETA 1, etc. 
#10min

In [ ]:
#LEER MODELO Y RECONOCER ROSTROS

import cv2 as cv
import os 

faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.read(r'C:\Users\rovega\Desktop\ITM\Sem 9\Inteligencia Artificial\Git IA - Actividades\ModelosXML\Eigenface-carasSentimientos.xml') #lo puedo usar en otros programas. Una vez entrenado el resultado es instantaneo.

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier(r'C:\Users\rovega\Desktop\ITM\Sem 9\Inteligencia Artificial\Git IA - Actividades\ModelosXML\haarcascade_frontalface_alt.xml') #uso clasificador, para detectar rostro dentro de escena, barre 
while True:
    ret, frame = cap.read() 
    if ret == False: break #si no hay camara, rompo proceso
        
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) 
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3) #varios reconocimientos
    
    for(x, y, w, h) in rostros:
        
        frame2 = cpGray[y:y+h, x:x+w] #recorto cara
        frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2) #paso frames/recortes al modelo, si detecta la cara que le mande, me regresa un rango de valor entre las caras, 
        #CON HAARCASCADE TOMO RECORTE DE TU CARA, y dime si en tu base tienes estos valores. Cada rostros tiene su clase 0,1 (persona), 
        
        #cv.putText(frame, '{}'.format(result), (x,y-20), 1,3.3, (255,255,0), 1, cv.LINE_AA) #IMPRIMIR VALOR DE RESULT
        #SI EL RANGO ES MAS ALTO, ES LA CONFIANZA Q TENGO PARA RECONOCER PERSONAS, SI VALOR ES 7,000 (fotos son suficientes para poder reconocer). RANGO ES DE 0-1, DE 1000 A 10,000
        if result[1] > 2800: #si el rango es mayor a 2800 (valor q es usualmente mayor a 2800), es LALO, si no es DESCONOCIDO. DA VALOR "(0, 4932)" 0 define clase de la cara (ROY),  clase 1 (LALO), clase 2 (PEPE)
            cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA) #PONER TEXTO DE ETIQUETA, y coordenadas 
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2) #PINTA RECTANGULO
        else:
            cv.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
            
    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    
    if k == 27:
        break
        
cap.release()
cv.destroyAllWindows()